In [1]:
import tensorflow as tf
import keras
# Define the input shape
input_shape = (224, 224, 3)

mod = keras.models.load_model("C:/Users/Charanteja/OneDrive/Pictures/Desktop/B.Tech_Final_Year_Project/har_inceptionnet.h5", compile= False,custom_objects={'input_shape': input_shape})


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

person_detector = YOLO("yolov8n.pt")
# res = person_detector("D:\sampled_frames\kicking\S1_kicking_toRight_HD\Images\S1_kicking_toRight_HD_0.jpg",classes=0)


classes = ['clapping','hitting','kicking','punching','running','stabbing','walking','waving_hands']
cam2 = cv2.VideoCapture("C:/Users/Charanteja/uav_actions_train/train_set/walking_f_b/S10_walking_frontView_HD.mp4")
while cam2.isOpened():
        
    ret,image = cam2.read()
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = person_detector(image_rgb,classes=0)
    
    for detections in results:
        bbox = detections.boxes.xyxy
        bbox_ = np.array(bbox)
        annot = list([int(bbox_[0][i]) for i in (range(len(bbox_[0])))])
        x1,y1,x2,y2 = annot[0],annot[1],annot[2],annot[3]
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        # color = (0,255,0)
        p1 = (x1,y1)
        p2 = (x2,y2)
        cv2.rectangle(image_rgb, p1, p2, (0, 255, 255), 2)
        person = image_rgb[y1:y2,x1:x2]
        person = cv2.resize(person,(224,224))
        person = person/255.0
        person = person.reshape(224,224,3)
        person = np.expand_dims(person,axis=0)
        pred = mod.predict(person)
        predicted_class = np.argmax(pred, axis=1)
        label = classes[int(predicted_class)]
        #print(label)
        # plt.imshow(image_rgb)
        # plt.show()
        # break
        cv2.putText(image_rgb, label,(x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 8)
        image_rgb=cv2.resize(image_rgb,(800,600))
        image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
        cv2.imshow('Detected persons', image_rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam2.release()
cv2.destroyAllWindows()

# Replace 'path_to_image' with the path to your image file